# “dx extract_dataset” in R
<hr/>
***As-Is Software Disclaimer***

This content in this repository is delivered “As-Is”. Notwithstanding anything to the contrary, DNAnexus will have no warranty, support, liability or other obligations with respect to Materials provided hereunder.

<hr/>

This notebook demonstrates usage of the dx command `extract_dataset` for:
* Retrieval of Apollo-stored data, as referenced within entities and fields of a Dataset or Cohort object on the platform
* Retrieval of the underlying data dictionary files used to generate a Dataset object on the platform

<a href="https://github.com/dnanexus/OpenBio/blob/master/LICENSE.md">MIT License</a> applies to this notebook.

## Preparing your environment
### Launch spec:

* App name: JupyterLab with Python, R, Stata, ML ()
* Kernel: R
* Instance type: Spark Cluster - mem1_ssd1_v2_x2, 3 nodes 
* Snapshot: `/.Notebook_snapshots/jupyter_snapshot.gz`
* Cost: < $0.2
* Runtime: =~ 10 min
* Data description: Input for this notebook is a v3.0 Dataset or Cohort object ID

### Install dxpy
extract_dataset requires dxpy version >= 0.329.0. If running the command from your local environment (i.e. off of the DNAnexus platform), it may be required to also install pandas. For example, pip3 install -U dxpy[pandas]

In [ ]:
system("pip3 show dxpy", intern = TRUE)

### Install tidyverse for data processing

Quick note - you will need to read the licenses for the tidyverse in order to make sure whether you and your group are comfortable with the licensing terms.

The `{xvhelper}` package is a small package that helps you translate datasets that contain raw values to coded values from the data dictionary and codings files (see below).

In [1]:
install.packages(c("readr", "stringr", "dplyr", "glue", "reactable", "janitor", "remotes"))
remotes::install_github("laderast/xvhelper")

Installing packages into ‘/usr/local/lib/R/site-library’
(as ‘lib’ is unspecified)

also installing the dependencies ‘bit’, ‘bit64’, ‘progress’, ‘hms’, ‘vroom’, ‘tzdb’, ‘reactR’, ‘snakecase’





vctrs      (0.4.2 -> 0.5.1) [CRAN]
timechange (NA    -> 0.1.1) [CRAN]
lubridate  (1.8.0 -> 1.9.0) [CRAN]


Installing 3 packages: vctrs, timechange, lubridate

Installing packages into ‘/usr/local/lib/R/site-library’
(as ‘lib’ is unspecified)



* checking for file ‘/tmp/RtmplI8sMk/remotes7f212b9cee/laderast-xvhelper-d41baac/DESCRIPTION’ ... OK
* preparing ‘xvhelper’:
* checking DESCRIPTION meta-information ... OK
* checking for LF line-endings in source and make files and shell scripts
* checking for empty or unneeded directories
* building ‘xvhelper_0.0.0.9000.tar.gz’



Installing package into ‘/usr/local/lib/R/site-library’
(as ‘lib’ is unspecified)



### Import packages

In [2]:
library(dplyr)
library(readr)
library(stringr)
library(glue)
library(reactable)
library(xvhelper)


Attaching package: ‘dplyr’


The following objects are masked from ‘package:stats’:

    filter, lag


The following objects are masked from ‘package:base’:

    intersect, setdiff, setequal, union




### 1. Assign environment variables

In [3]:
# The referenced Dataset is private and provided only to demonstrate an example input. The user will need to supply a permissible and valid record-id
# Assign project-id of dataset
# Assign dataset record-id
rid <- 'record-GFx5PX8JY63f9QPZBBJZ547x'
# Assign joint dataset project-id:record-id
dataset <- rid

### 2. Call “dx extract_dataset” using a supplied dataset

We'll use the `{glue}` package to put our bash commands together for `dx extract_dataset`, and use `system()` to execute our bash code.

`glue::glue()` has the advantage of not needing to `paste()` together strings. The string substitution is cleaner.

In [4]:
cmd <- glue::glue("dx extract_dataset {dataset} -ddd")

cmd

dx extract_dataset record-GFx5PX8JY63f9QPZBBJZ547x -ddd

Let's execute our command using `system()` and then we will list the files that result using `list.files()`. We generate three files in the directory in JupyterLab storage:

- *dataset_name*`.codings.csv`
- *dataset_name*`.data_dictionary.csv`
- *dataset_name*`.entity_dictionary.csv`

In [5]:
system(cmd)
list.files()

[1] "hypertension_cases.codings.csv"          
[2] "hypertension_cases.data_dictionary.csv"  
[3] "hypertension_cases.entity_dictionary.csv"

#### Preview data in the three dictionary (*.csv) files

In [ ]:
#codings_file <- system("ls *.codings.csv", intern = TRUE)
codings_file <- list.files(pattern="*.codings.csv")
codings_df <- read_csv(codings_file, show_col_types = FALSE)
head(codings_df)

In [ ]:
entity_dict_file <- system("ls *.entity_dictionary.csv", intern=TRUE)
entity_dict_df <- read_csv(entity_dict_file, show_col_types = FALSE)
head(entity_dict_df)

### Understanding the Data Dictionary File

The data dictionary is the glue for the entire dataset. It maps:

- Entity to Fields
- Fields to Codings
- Entity to Entity

We'll use the data dictionary to understand how to building our list of fields, and later, we'll join it to the codings file to build a list of fields and their coded values.

There are more columns to the data dictionary, but let's first see the `entity`, `name`, `title`, and `type` columns:

In [ ]:
#data_dict_file <- system("ls *.data_dictionary.csv", intern=TRUE)
data_dict_file <- list.files(pattern="*.data_dictionary.csv")
data_dict_df <- readr::read_csv(data_dict_file, show_col_types = FALSE)
data_dict_df <- data_dict_df 

data_dict_df %>%
        select(entity, name, title, type) %>%
        head()


### 3. Parse returned metadata and extract entity/field names

Let's search for some fields. We want the following fields:

- `Coffee intake | instance 0`
- `Sex` (Gender)
- `Smoked cigarette or pipe within last hour | Instance 0`

We can use the `{reactable}` package to make a searchable table of the data dictionary. This will help in finding fields.

Note the search box in the top right of the table - when we have many fields, we can use the search box to find fields of interest. Try searching for `Coffee intake` and see what fields pop up.

In [ ]:
data_dict_df <- data_dict_df %>%
    relocate(name, title) %>%
    mutate(ent_field = glue::glue("{entity}.{name}"))

basic_data_dict <- data_dict_df |>
                    select(title, name, entity, ent_field, coding_name, is_multi_select, is_sparse_coding)

reactable::reactable(basic_data_dict, searchable = TRUE)

Another strategy for searching fields: we can use `grepl` within `dplyr::filter()` to search for fields that match our criteria.

Note we're chaining the `grepl` statements with an OR `|`.

We're also concatenating `entity` and `name` to a new variable, `ent_field`, which we'll use when we specify our list of fields.

In [9]:
filtered_dict <- data_dict_df |>
    filter(grepl("Coffee type", title) | 
           grepl("Sex", title) | 
           grepl("Smoked", title) | 
           grepl("Age at recruitment", title) |
           grepl("Diagnoses - main ICD10", title) 
          ) |>
    filter(entity == "participant") |>
    arrange(title) 

filtered_dict %>%
    select(name, title, ent_field)

name,title,ent_field
<chr>,<chr>,<glue>
p21022,Age at recruitment,participant.p21022
p1508_i0,Coffee type | Instance 0,participant.p1508_i0
p1508_i1,Coffee type | Instance 1,participant.p1508_i1
p1508_i2,Coffee type | Instance 2,participant.p1508_i2
p1508_i3,Coffee type | Instance 3,participant.p1508_i3
p41202,Diagnoses - main ICD10,participant.p41202
p31,Sex,participant.p31
p22019,Sex chromosome aneuploidy,participant.p22019
p22022,Sex inference X probe-intensity,participant.p22022


Let's use this subset of fields - we'll pull the `ent_field` column, and paste it together into a single comma delimited string using `paste`:

In [10]:
field_list <- filtered_dict %>%
    pull(ent_field)

#field_list <- field_list[200:210]
field_list <- paste(field_list, collapse = ",")
field_list

[1] "participant.p21022,participant.p1508_i0,participant.p1508_i1,participant.p1508_i2,participant.p1508_i3,participant.p41202,participant.p31,participant.p22019,participant.p22022,participant.p22023,participant.p41226,participant.p20524,participant.p20490,participant.p3159_i0,participant.p3159_i1,participant.p3159_i2,participant.p3159_i3"

### 4. Use extracted entity and field names as input to the called function, “dx extract_dataset” and extract data

Again, we'll use `glue()` here for cleaner string substitution.

We'll extract the cohort information to a file called `cohort_data.csv` and work with this file for the rest of the notebook.

In [11]:
cohort <- dataset
cohort_template <- "dx extract_dataset {cohort} --fields {field_list} -o cohort_data.csv"
cmd <- glue::glue(cohort_template)

cmd

system(cmd)

dx extract_dataset record-GFx5PX8JY63f9QPZBBJZ547x --fields participant.p21022,participant.p1508_i0,participant.p1508_i1,participant.p1508_i2,participant.p1508_i3,participant.p41202,participant.p31,participant.p22019,participant.p22022,participant.p22023,participant.p41226,participant.p20524,participant.p20490,participant.p3159_i0,participant.p3159_i1,participant.p3159_i2,participant.p3159_i3 -o cohort_data.csv

#### Preview data in the retrieved data file

We'll see that the retrieved data contains the integer and character codes. These must be decoded (see below):

In [12]:
data_df <- readr::read_csv("cohort_data.csv", show_col_types = FALSE)
head(data_df)

participant.p21022,participant.p1508_i0,participant.p1508_i1,participant.p1508_i2,participant.p1508_i3,participant.p41202,participant.p31,participant.p22019,participant.p22022,participant.p22023,participant.p41226,participant.p20524,participant.p20490,participant.p3159_i0,participant.p3159_i1,participant.p3159_i2,participant.p3159_i3
<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<chr>,<dbl>,<lgl>,<dbl>,<dbl>,<chr>,<dbl>,<dbl>,<dbl>,<lgl>,<dbl>,<dbl>
45,3,NA,NA,NA,"[""C509"",""G560"",""K573"",""K620"",""O021"",""O800"",""T810""]",0,NA,251.62,118.83,[2],NA,NA,NA,NA,NA,NA
61,2,NA,NA,NA,"[""D649"",""H269"",""I200"",""I209"",""I210"",""K449"",""K801"",""M179"",""N812"",""R074"",""R104""]",0,NA,658.53,288.65,NA,NA,NA,0,NA,NA,NA
63,2,NA,NA,NA,"[""E86"",""H251"",""H811"",""J101"",""J181"",""J441"",""K564"",""K565"",""K590"",""L989"",""M5499"",""N320"",""R33"",""Z466""]",1,NA,356.06,401.10,NA,NA,NA,0,NA,NA,NA
64,1,NA,NA,NA,"[""C64"",""D638"",""E112"",""I10"",""I495"",""J188"",""K635"",""N184"",""N185"",""R074"",""R55"",""R91"",""S008"",""S099""]",1,NA,1133.11,1300.66,NA,0,0,0,NA,NA,NA
64,3,NA,NA,NA,"[""D223"",""G119"",""H269"",""J22"",""L570"",""N390""]",0,NA,1018.51,445.07,NA,NA,NA,NA,NA,NA,NA
51,1,NA,NA,NA,"[""K573"",""N393"",""N604"",""N811"",""N814"",""Z302"",""Z530""]",0,NA,408.86,200.47,NA,NA,NA,NA,NA,NA,NA


## Decoding columns with xvhelper

xvhelper is a little R package that will return the actual values of the returned data.

To use it, you build a `coded_col_df` using `merge_coding_data_dict()` and then translate the categorical columns to values using `decode_categories()`, and then change the column names to R friendly clean ones using `decode_column_names()`.

Note that we need to run `decode_df()` before we run `decode_category()`

In [3]:
#install via remotes::install_github()
#install.packages("remotes")
remotes::install_github("laderast/xvhelper")

library(xvhelper)

Skipping install of 'xvhelper' from a github remote, the SHA1 (d41baacf) has not changed since last install.
  Use `force = TRUE` to force installation



In [19]:
data_dict <- readr::read_csv("hypertension_cases.data_dictionary.csv")
  coding_dict <- readr::read_csv("hypertension_cases.codings.csv")

coded_col_df <- xvhelper::merge_coding_data_dict(coding_dict = coding_dict, data_dict = data_dict)

data_fields <- colnames(data_df)
coded_col_df2 <- coded_col_df |>
    dplyr::filter(ent_field %in% data_fields)

decoded <- data_df |>
    xvhelper::decode_single(coded_col_df2) 

    
head(decoded)

Warning message:
“One or more parsing issues, call `problems()` on your data frame for details,
e.g.:
  dat <- vroom(...)
  problems(dat)”
Rows: 25763 Columns: 16
── Column specification ────────────────────────────────────────────────────────
Delimiter: ","
chr (11): entity, name, type, primary_key_type, coding_name, folder_path, is...
lgl  (5): concept, description, longitudinal_axis_type, referenced_entity_fi...

ℹ Use `spec()` to retrieve the full column specification for this data.
ℹ Specify the column types or set `show_col_types = FALSE` to quiet this message.
Rows: 88736 Columns: 6
── Column specification ────────────────────────────────────────────────────────
Delimiter: ","
chr (4): coding_name, code, meaning, parent_code
dbl (1): display_order
lgl (1): concept

ℹ Use `spec()` to retrieve the full column specification for this data.
ℹ Specify the column types or set `show_col_types = FALSE` to quiet this message.


participant.p21022,participant.p1508_i0,participant.p1508_i1,participant.p1508_i2,participant.p1508_i3,participant.p41202,participant.p31,participant.p22019,participant.p22022,participant.p22023,participant.p41226,participant.p20524,participant.p20490,participant.p3159_i0,participant.p3159_i1,participant.p3159_i2,participant.p3159_i3
<dbl>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<dbl>,<dbl>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>
45,"Ground coffee (include espresso, filter etc)",NA,NA,NA,"[""C509"",""G560"",""K573"",""K620"",""O021"",""O800"",""T810""]",Female,NA,251.62,118.83,[2],NA,NA,NA,NA,NA,NA
61,Instant coffee,NA,NA,NA,"[""D649"",""H269"",""I200"",""I209"",""I210"",""K449"",""K801"",""M179"",""N812"",""R074"",""R104""]",Female,NA,658.53,288.65,NA,NA,NA,No,NA,NA,NA
63,Instant coffee,NA,NA,NA,"[""E86"",""H251"",""H811"",""J101"",""J181"",""J441"",""K564"",""K565"",""K590"",""L989"",""M5499"",""N320"",""R33"",""Z466""]",Male,NA,356.06,401.10,NA,NA,NA,No,NA,NA,NA
64,Decaffeinated coffee (any type),NA,NA,NA,"[""C64"",""D638"",""E112"",""I10"",""I495"",""J188"",""K635"",""N184"",""N185"",""R074"",""R55"",""R91"",""S008"",""S099""]",Male,NA,1133.11,1300.66,NA,Never true,Never true,No,NA,NA,NA
64,"Ground coffee (include espresso, filter etc)",NA,NA,NA,"[""D223"",""G119"",""H269"",""J22"",""L570"",""N390""]",Female,NA,1018.51,445.07,NA,NA,NA,NA,NA,NA,NA
51,Decaffeinated coffee (any type),NA,NA,NA,"[""K573"",""N393"",""N604"",""N811"",""N814"",""Z302"",""Z530""]",Female,NA,408.86,200.47,NA,NA,NA,NA,NA,NA,NA


Next, we'll decode the columns that have multiple values (such as `participant.p41202`, which is `Diagnosis - main ICD10`. 

This cohort is very large, so note that decoding these columns is going to take some time.

In [15]:
decoded2 <- decoded |>
    decode_multi(coded_col_df2)

head(decoded2)

participant.p21022,participant.p1508_i0,participant.p1508_i1,participant.p1508_i2,participant.p1508_i3,participant.p41202,participant.p31,participant.p22019,participant.p22022,participant.p22023,participant.p41226,participant.p20524,participant.p20490,participant.p3159_i0,participant.p3159_i1,participant.p3159_i2,participant.p3159_i3
<dbl>,<chr>,<chr>,<chr>,<dbl>,<chr>,<chr>,<lgl>,<dbl>,<dbl>,<chr>,<dbl>,<dbl>,<chr>,<chr>,<chr>,<dbl>
45,"Ground coffee (include espresso, filter etc)",NA,NA,NA,"C50.9 Breast, unspecified, G56.0 Carpal tunnel syndrome, K57.3 Diverticular disease of large intestine without perforation or abscess, K62.0 Anal polyp, O02.1 Missed abortion, O80.0 Spontaneous vertex delivery, T81.0 Haemorrhage and haematoma complicating a procedure, not elsewhere classified",Female,NA,251.62,118.83,Female,NA,NA,NA,NA,NA,NA
61,Instant coffee,NA,NA,NA,"D64.9 Anaemia, unspecified, H26.9 Cataract, unspecified, I20.0 Unstable angina, I20.9 Angina pectoris, unspecified, I21.0 Acute transmural myocardial infarction of anterior wall, K44.9 Diaphragmatic hernia without obstruction or gangrene, K80.1 Calculus of gallbladder with other cholecystitis, M17.9 Gonarthrosis, unspecified, N81.2 Incomplete uterovaginal prolapse, R07.4 Chest pain, unspecified, R10.4 Other and unspecified abdominal pain",Female,NA,658.53,288.65,NA,NA,NA,No,NA,NA,NA
63,Instant coffee,NA,NA,NA,"E86 Volume depletion, H25.1 Senile nuclear cataract, H81.1 Benign paroxysmal vertigo, J10.1 Influenza with other respiratory manifestations, influenza virus identified, J18.1 Lobar pneumonia, unspecified, J44.1 Chronic obstructive pulmonary disease with acute exacerbation, unspecified, K56.4 Other impaction of intestine, K56.5 Intestinal adhesions [bands] with obstruction, K59.0 Constipation, L98.9 Disorder of skin and subcutaneous tissue, unspecified, M54.99 Dorsalgia, unspecified (Site unspecified), N32.0 Bladder-neck obstruction, R33 Retention of urine, Z46.6 Fitting and adjustment of urinary device",Male,NA,356.06,401.10,NA,NA,NA,No,NA,NA,NA
64,Decaffeinated coffee (any type),NA,NA,NA,"C64 Malignant neoplasm of kidney, except renal pelvis, D63.8 Anaemia in other chronic diseases classified elsewhere, E11.2 With renal complications, I10 Essential (primary) hypertension, I49.5 Sick sinus syndrome, J18.8 Other pneumonia, organism unspecified, K63.5 Polyp of colon, N18.4 Chronic kidney disease, stage 4, N18.5 Chronic kidney disease, stage 5, R07.4 Chest pain, unspecified, R55 Syncope and collapse, R91 Abnormal findings on diagnostic imaging of lung, S00.8 Superficial injury of other parts of head, S09.9 Unspecified injury of head",Male,NA,1133.11,1300.66,NA,0,0,No,NA,NA,NA
64,"Ground coffee (include espresso, filter etc)",NA,NA,NA,"D22.3 Melanocytic naevi of other and unspecified parts of face, G11.9 Hereditary ataxia, unspecified, H26.9 Cataract, unspecified, J22 Unspecified acute lower respiratory infection, L57.0 Actinic keratosis, N39.0 Urinary tract infection, site not specified",Female,NA,1018.51,445.07,NA,NA,NA,NA,NA,NA,NA
51,Decaffeinated coffee (any type),NA,NA,NA,"K57.3 Diverticular disease of large intestine without perforation or abscess, N39.3 Stress incontinence, N60.4 Mammary duct ectasia, N81.1 Cystocele, N81.4 Uterovaginal prolapse, unspecified, Z30.2 Sterilisation, Z53.0 Procedure not carried out because of contraindication",Female,NA,408.86,200.47,NA,NA,NA,NA,NA,NA,NA


In [24]:
decoded3 <- decoded2 |>
    decode_column_names(coded_col_df2)

In [26]:
write.csv(decoded3, file="cohort_decoded.csv")

### Save Output to Project

In [27]:
system("dx upload *.csv --destination /user/tladeras/")

# Using Spark/`{sparklyr}` to extract datasets

If our query is takes longer than 2 minutes to execute, `dx extract_datasets` will not run. In that case, we will start up a Spark instance of JupyterLab and run the SparkSQL directly in the notebook.

1. Extract the relevant dictionary information
2. Connect to our Spark Cluster using `sparklyr`
3. Extract the dataset SQL using the `--sql` option with `dx extract_dataset` and save output to a file
4. Load query from file, clean the SQL
5. Use `dbGetQuery` to load the dataset as a `data.frame`
6. Use `readr::write_csv()` to save `data.frame` as CSV file
7. Upload our CSV file back to project storage using `dx upload`.

In [9]:
install.packages("sparklyr")


Installing package into ‘/usr/local/lib/R/site-library’
(as ‘lib’ is unspecified)



## Generate SQL from `dx extract_dataset`

If we use the `--sql` flag, the query will not execute, but the SparkSQL needed to execute the query will be saved. 

We'll use the `-o` option to save our query as `cohort.sql`.

In [3]:
system("dx extract_dataset record-GFx5PX8JY63f9QPZBBJZ547x -ddd")

In [10]:
field_list <- 'participant.eid,participant.p21022,participant.p1508_i0,participant.p1508_i1,participant.p1508_i2,participant.p41202,participant.p31,participant.p41226,participant.p3159_i0,participant.p3159_i1,participant.p3159_i2'

cohort <- "record-GFx5PX8JY63f9QPZBBJZ547x"

cohort_template <- glue::glue("dx extract_dataset {cohort} --fields {field_list} --sql -o cohort.sql")

cohort_template

system(cohort_template)
list.files()

dx extract_dataset record-GFx5PX8JY63f9QPZBBJZ547x --fields participant.eid,participant.p21022,participant.p1508_i0,participant.p1508_i1,participant.p1508_i2,participant.p41202,participant.p31,participant.p41226,participant.p3159_i0,participant.p3159_i1,participant.p3159_i2 --sql -o cohort.sql

[1] "cohort.sql"                              
[2] "hypertension_cases.codings.csv"          
[3] "hypertension_cases.data_dictionary.csv"  
[4] "hypertension_cases.entity_dictionary.csv"

## Connect to Spark Cluster

The first thing we need to do is connect to the Spark Cluster. We'll use the `{sparklyr}` and `{DBI}` packages to connect.

Make sure to only connect once - Spark will throw an error if you try to connect twice by rerunning this code block.

To fix this, you will need to restart the kernel using **Kernel > Restart Kernel*

In [11]:
library(sparklyr)
library(DBI)
port <- Sys.getenv("SPARK_MASTER_PORT")
master <- paste("spark://master:", port, sep = '')
sc = spark_connect(master)


Attaching package: ‘sparklyr’


The following object is masked from ‘package:stats’:

    filter




## Load SQL from file

We'll read the `cohort.sql` file using `read_file()` and then execute the SQL using `dbGetQuery()`.

Here's what the actual query looks like:

In [12]:
library(readr)
retrieve_sql <-read_file("cohort.sql")
retrieve_sql <- gsub("[;\n]", "", retrieve_sql)
retrieve_sql

[1] "SELECT `participant_0001_1`.`eid` AS `participant.eid`, `participant_0042_1`.`p21022` AS `participant.p21022`, `participant_0005_1`.`p1508_i0` AS `participant.p1508_i0`, `participant_0005_1`.`p1508_i1` AS `participant.p1508_i1`, `participant_0005_1`.`p1508_i2` AS `participant.p1508_i2`, `participant_0074_1`.`p41202` AS `participant.p41202`, `participant_0001_1`.`p31` AS `participant.p31`, `participant_0074_1`.`p41226` AS `participant.p41226`, `participant_0007_1`.`p3159_i0` AS `participant.p3159_i0`, `participant_0007_1`.`p3159_i1` AS `participant.p3159_i1`, `participant_0007_1`.`p3159_i2` AS `participant.p3159_i2` FROM `database_g8zjpv0jy8jk9k1vpb4bk8qz__app46926_20220310050548`.`participant_0001` AS `participant_0001_1` LEFT OUTER JOIN `database_g8zjpv0jy8jk9k1vpb4bk8qz__app46926_20220310050548`.`participant_0042` AS `participant_0042_1` ON `participant_0001_1`.`eid` = `participant_0042_1`.`eid` LEFT OUTER JOIN `database_g8zjpv0jy8jk9k1vpb4bk8qz__app46926_20220310050548`.`participant_0005` AS `participant_0005_1` ON `participant_0001_1`.`eid` = `participant_0005_1`.`eid` LEFT OUTER JOIN `database_g8zjpv0jy8jk9k1vpb4bk8qz__app46926_20220310050548`.`participant_0074` AS `participant_0074_1` ON `participant_0001_1`.`eid` = `participant_0074_1`.`eid` LEFT OUTER JOIN `database_g8zjpv0jy8jk9k1vpb4bk8qz__app46926_20220310050548`.`participant_0007` AS `participant_0007_1` ON `participant_0001_1`.`eid` = `participant_0007_1`.`eid` WHERE `participant_0001_1`.`eid` IN (SELECT DISTINCT `participant_0001_1`.`eid` AS `eid` FROM `database_g8zjpv0jy8jk9k1vpb4bk8qz__app46926_20220310050548`.`participant_0001` AS `participant_0001_1` LEFT OUTER JOIN `database_g8zjpv0jy8jk9k1vpb4bk8qz__app46926_20220310050548`.`participant_0075` AS `participant_0075_1` ON `participant_0001_1`.`eid` = `participant_0075_1`.`eid` WHERE ARRAY_CONTAINS(`participant_0075_1`.`dnx_p41270_hierarchy`, 'Block I10-I15'))"

## Execute Our Query with DBI/sparklyr

Now we can execute our SQL with `dbGetQuery()`. Note we utilize our Spark Connection (`sc`) to execute it.

Note that the `data.frame` returned by `dbGetQuery()` is not identical to the `.csv` file generated by `dx extract_dataset`. 

`dbGetQuery()` returns columns that have multiple values per lines as list columns. `{xvhelper}` can transform these columns into comma separated strings.

In [13]:
df <- dbGetQuery(sc, retrieve_sql)
head(df)

,participant.eid,participant.p21022,participant.p1508_i0,participant.p1508_i1,participant.p1508_i2,participant.p41202,participant.p31,participant.p41226,participant.p3159_i0,participant.p3159_i1,participant.p3159_i2
,<chr>,<dbl>,<dbl>,<dbl>,<dbl>,<list>,<dbl>,<list>,<dbl>,<dbl>,<dbl>
1,1000083,45,3,NA,NA,"C509, G560, K573, K620, O021, O800, T810",0,2,NA,NA,NA
2,1000150,61,2,NA,NA,"D649, H269, I200, I209, I210, K449, K801, M179, N812, R074, R104",0,NA,0,NA,NA
3,1000168,63,2,NA,NA,"E86 , H251 , H811 , J101 , J181 , J441 , K564 , K565 , K590 , L989 , M5499, N320 , R33 , Z466",1,NA,0,NA,NA
4,1000738,64,1,NA,NA,"C64 , D638, E112, I10 , I495, J188, K635, N184, N185, R074, R55 , R91 , S008, S099",1,NA,0,NA,NA
5,1000855,64,3,NA,NA,"D223, G119, H269, J22 , L570, N390",0,NA,NA,NA,NA
6,1000962,51,1,NA,NA,"K573, N393, N604, N811, N814, Z302, Z530",0,NA,NA,NA,NA


In [17]:
library(xvhelper)
    
  data_dict <- readr::read_csv("hypertension_cases.data_dictionary.csv", show_col_types=FALSE)
  coding_dict <- readr::read_csv("hypertension_cases.codings.csv", show_col_types=FALSE)
    

merged_dict <- xvhelper::merge_coding_data_dict(coding_dict, data_dict)

decoded <- df |> 
 decode_single(merged_dict) |>
 decode_multi(merged_dict)


Warning message:
“One or more parsing issues, call `problems()` on your data frame for details,
e.g.:
  dat <- vroom(...)
  problems(dat)”


In [18]:
head(decoded)

,participant.eid,participant.p21022,participant.p1508_i0,participant.p1508_i1,participant.p1508_i2,participant.p41202,participant.p31,participant.p41226,participant.p3159_i0,participant.p3159_i1,participant.p3159_i2
,<chr>,<dbl>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>
1,1000083,45,"Ground coffee (include espresso, filter etc)",NA,NA,"C50.9 Breast, unspecified, G56.0 Carpal tunnel syndrome, K57.3 Diverticular disease of large intestine without perforation or abscess, K62.0 Anal polyp, O02.1 Missed abortion, O80.0 Spontaneous vertex delivery, T81.0 Haemorrhage and haematoma complicating a procedure, not elsewhere classified",Female,Female,NA,NA,NA
2,1000150,61,Instant coffee,NA,NA,"D64.9 Anaemia, unspecified, H26.9 Cataract, unspecified, I20.0 Unstable angina, I20.9 Angina pectoris, unspecified, I21.0 Acute transmural myocardial infarction of anterior wall, K44.9 Diaphragmatic hernia without obstruction or gangrene, K80.1 Calculus of gallbladder with other cholecystitis, M17.9 Gonarthrosis, unspecified, N81.2 Incomplete uterovaginal prolapse, R07.4 Chest pain, unspecified, R10.4 Other and unspecified abdominal pain",Female,NA,No,NA,NA
3,1000168,63,Instant coffee,NA,NA,"E86 Volume depletion, H25.1 Senile nuclear cataract, H81.1 Benign paroxysmal vertigo, J10.1 Influenza with other respiratory manifestations, influenza virus identified, J18.1 Lobar pneumonia, unspecified, J44.1 Chronic obstructive pulmonary disease with acute exacerbation, unspecified, K56.4 Other impaction of intestine, K56.5 Intestinal adhesions [bands] with obstruction, K59.0 Constipation, L98.9 Disorder of skin and subcutaneous tissue, unspecified, M54.99 Dorsalgia, unspecified (Site unspecified), N32.0 Bladder-neck obstruction, R33 Retention of urine, Z46.6 Fitting and adjustment of urinary device",Male,NA,No,NA,NA
4,1000738,64,Decaffeinated coffee (any type),NA,NA,"C64 Malignant neoplasm of kidney, except renal pelvis, D63.8 Anaemia in other chronic diseases classified elsewhere, E11.2 With renal complications, I10 Essential (primary) hypertension, I49.5 Sick sinus syndrome, J18.8 Other pneumonia, organism unspecified, K63.5 Polyp of colon, N18.4 Chronic kidney disease, stage 4, N18.5 Chronic kidney disease, stage 5, R07.4 Chest pain, unspecified, R55 Syncope and collapse, R91 Abnormal findings on diagnostic imaging of lung, S00.8 Superficial injury of other parts of head, S09.9 Unspecified injury of head",Male,NA,No,NA,NA
5,1000855,64,"Ground coffee (include espresso, filter etc)",NA,NA,"D22.3 Melanocytic naevi of other and unspecified parts of face, G11.9 Hereditary ataxia, unspecified, H26.9 Cataract, unspecified, J22 Unspecified acute lower respiratory infection, L57.0 Actinic keratosis, N39.0 Urinary tract infection, site not specified",Female,NA,NA,NA,NA
6,1000962,51,Decaffeinated coffee (any type),NA,NA,"K57.3 Diverticular disease of large intestine without perforation or abscess, N39.3 Stress incontinence, N60.4 Mammary duct ectasia, N81.1 Cystocele, N81.4 Uterovaginal prolapse, unspecified, Z30.2 Sterilisation, Z53.0 Procedure not carried out because of contraindication",Female,NA,NA,NA,NA


## Decode Column Names

We can also decode the column names:

In [19]:
decoded_cols <- decoded |>
    xvhelper::decode_column_names(merged_dict)

head(decoded_cols)

,participant_id,age_at_recruitment,coffee_type_instance_0,coffee_type_instance_1,coffee_type_instance_2,diagnoses_main_icd10,sex,sex_of_baby,smoked_cigarette_or_pipe_within_last_hour_instance_0,smoked_cigarette_or_pipe_within_last_hour_instance_1,smoked_cigarette_or_pipe_within_last_hour_instance_2
,<chr>,<dbl>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>
1,1000083,45,"Ground coffee (include espresso, filter etc)",NA,NA,"C50.9 Breast, unspecified, G56.0 Carpal tunnel syndrome, K57.3 Diverticular disease of large intestine without perforation or abscess, K62.0 Anal polyp, O02.1 Missed abortion, O80.0 Spontaneous vertex delivery, T81.0 Haemorrhage and haematoma complicating a procedure, not elsewhere classified",Female,Female,NA,NA,NA
2,1000150,61,Instant coffee,NA,NA,"D64.9 Anaemia, unspecified, H26.9 Cataract, unspecified, I20.0 Unstable angina, I20.9 Angina pectoris, unspecified, I21.0 Acute transmural myocardial infarction of anterior wall, K44.9 Diaphragmatic hernia without obstruction or gangrene, K80.1 Calculus of gallbladder with other cholecystitis, M17.9 Gonarthrosis, unspecified, N81.2 Incomplete uterovaginal prolapse, R07.4 Chest pain, unspecified, R10.4 Other and unspecified abdominal pain",Female,NA,No,NA,NA
3,1000168,63,Instant coffee,NA,NA,"E86 Volume depletion, H25.1 Senile nuclear cataract, H81.1 Benign paroxysmal vertigo, J10.1 Influenza with other respiratory manifestations, influenza virus identified, J18.1 Lobar pneumonia, unspecified, J44.1 Chronic obstructive pulmonary disease with acute exacerbation, unspecified, K56.4 Other impaction of intestine, K56.5 Intestinal adhesions [bands] with obstruction, K59.0 Constipation, L98.9 Disorder of skin and subcutaneous tissue, unspecified, M54.99 Dorsalgia, unspecified (Site unspecified), N32.0 Bladder-neck obstruction, R33 Retention of urine, Z46.6 Fitting and adjustment of urinary device",Male,NA,No,NA,NA
4,1000738,64,Decaffeinated coffee (any type),NA,NA,"C64 Malignant neoplasm of kidney, except renal pelvis, D63.8 Anaemia in other chronic diseases classified elsewhere, E11.2 With renal complications, I10 Essential (primary) hypertension, I49.5 Sick sinus syndrome, J18.8 Other pneumonia, organism unspecified, K63.5 Polyp of colon, N18.4 Chronic kidney disease, stage 4, N18.5 Chronic kidney disease, stage 5, R07.4 Chest pain, unspecified, R55 Syncope and collapse, R91 Abnormal findings on diagnostic imaging of lung, S00.8 Superficial injury of other parts of head, S09.9 Unspecified injury of head",Male,NA,No,NA,NA
5,1000855,64,"Ground coffee (include espresso, filter etc)",NA,NA,"D22.3 Melanocytic naevi of other and unspecified parts of face, G11.9 Hereditary ataxia, unspecified, H26.9 Cataract, unspecified, J22 Unspecified acute lower respiratory infection, L57.0 Actinic keratosis, N39.0 Urinary tract infection, site not specified",Female,NA,NA,NA,NA
6,1000962,51,Decaffeinated coffee (any type),NA,NA,"K57.3 Diverticular disease of large intestine without perforation or abscess, N39.3 Stress incontinence, N60.4 Mammary duct ectasia, N81.1 Cystocele, N81.4 Uterovaginal prolapse, unspecified, Z30.2 Sterilisation, Z53.0 Procedure not carried out because of contraindication",Female,NA,NA,NA,NA


## Working with Data in Spark

We can also load our data as a temporary table in Spark, which can be very useful when working with very large datasets.

This lets us leverage the power of Spark and the `{sparklyr}` package.

We'll first take our SQL query and use `CREATE TEMPORARY VIEW` to create a temporary View in Spark.

In [ ]:
temp_table_sql <- glue::glue("CREATE TEMPORARY VIEW pheno AS {retrieve_sql}")

result <- DBI::dbSendQuery(sc, temp_table_sql)
#sparklyr::sdf_register(temp_table_sql,sc)

Now we can register the data frame (`pheno`) as a Spark dataframe using `dplyr::tbl()`. 

Once we do that, we're able to leverage `sparklyr` to do `dplyr` like filtering:

In [ ]:
new_df <- dplyr::tbl(sc, "pheno")

filtered_df <- new_df |> dplyr::filter(`participant.p21022` > 50)

In [ ]:
head(filtered_df)

In [ ]:
colnames(new_df)

In [ ]:
new_df |> 
    dplyr::mutate(`participant.p31` = as.character(`participant.p31`)) |>
    dplyr::count(`participant.p31`)

#|>
#    dplyr::group_by(`participant.p31`) |> summarise(mean_age = mean(`participant.p21022`, na.rm=TRUE))

Once we're done with working with `filtered_df` in spark, we can pull it into memory using `collect()`. Then we can annotate the data as usual.

In [ ]:
in_memory <- collect(filtered_df)
head(in_memory)